## แบบฝึกหัดเขียนโปรแกรมครั้งที่ 6
### Loistic Regression

1. เขียนโปรแกรมสำหรับสร้างแบบจำลอง logistic regression ด้วยวิธี gradient descent สำหรับแก้ปัญหา logic AND, OR และ XOR (สำหรับปัญหา XOR ต้องใช้ interaction feature)

In [1]:
import numpy as np

## Sigmoid Function

สูตร sigmoid function 

$$
\sigma(z) = \frac{1}{1 + e^{-z}}
$$

## Cost Function 

สูตร cost function \( J(\theta) \) สำหรับ logistic regression 

$$
J(\theta) = -\frac{1}{m} \sum_{i=1}^{m} \left[ y^{(i)} \log(h_\theta(x^{(i)})) + (1 - y^{(i)}) \log(1 - h_\theta(x^{(i)})) \right]
$$

เมื่อ \( h_\theta(x) = \sigma(\theta^T x) \) เป็นค่าฟังก์ชันที่ใช้ sigmoid 

## Gradient Descent Function

สูตร update weight ด้วย Gradient descent 

$$
\theta := \theta - \frac{\alpha}{m} \sum_{i=1}^{m} \left( h_\theta(x^{(i)}) - y^{(i)} \right) x^{(i)}
$$




In [2]:
# Sigmoid function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Cost function for logistic regression
def logistic_cost_function(X, y, weights):
    m = len(y)
    h = sigmoid(np.dot(X, weights))
    cost = (-1/m) * (np.dot(y, np.log(h)) + np.dot((1-y), np.log(1-h)))
    return cost

# Gradient Descent function
def gradient_descent_binary(X, y, weights, learning_rate, iterations):
    m = len(y)
    cost_history = []
    
    for _ in range(iterations):
        h = sigmoid(np.dot(X, weights))
        weights -= (learning_rate / m) * np.dot(X.T, (h - y))
        cost = logistic_cost_function(X, y, weights)
        cost_history.append(cost)
    
    return weights, cost_history


- Prepare the Datasets for AND, OR, and XOR

In [3]:
# AND dataset
X_and = np.array([[1, 0, 0], [1, 0, 1], [1, 1, 0], [1, 1, 1]])
y_and = np.array([0, 0, 0, 1])

# OR dataset
X_or = np.array([[1, 0, 0], [1, 0, 1], [1, 1, 0], [1, 1, 1]])
y_or = np.array([0, 1, 1, 1])

# XOR dataset
X_xor = np.array([[1, 0, 0], [1, 0, 1], [1, 1, 0], [1, 1, 1]])
y_xor = np.array([0, 1, 1, 0])


- Train and Test the Model

In [4]:
# Initial weights
initial_weights = np.zeros(X_and.shape[1])

# Train AND Logic
weights_and, cost_history_and = gradient_descent_binary(X_and, y_and, initial_weights, learning_rate=0.1, iterations=1000)
print("AND Predictions:", sigmoid(np.dot(X_and, weights_and)) >= 0.5)

# Train OR Logic
weights_or, cost_history_or = gradient_descent_binary(X_or, y_or, initial_weights, learning_rate=0.1, iterations=1000)
print("OR Predictions:", sigmoid(np.dot(X_or, weights_or)) >= 0.5)

# Create interaction feature for XOR
X_xor_interaction = np.hstack([X_xor, (X_xor[:, 1] * X_xor[:, 2]).reshape(-1, 1)])

# Train XOR Logic
weights_xor, cost_history_xor = gradient_descent_binary(X_xor_interaction, y_xor, np.zeros(X_xor_interaction.shape[1]), learning_rate=0.1, iterations=1000)
print("XOR Predictions:", sigmoid(np.dot(X_xor_interaction, weights_xor)) >= 0.5)


AND Predictions: [False False False  True]
OR Predictions: [False  True  True  True]
XOR Predictions: [False  True  True False]
